<a href="https://colab.research.google.com/github/sydstewart/Orders/blob/main/Impsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Companion code to https://realpython.com/simulation-with-simpy/
'Simulating Real-World Processes With SimPy'
Python version: 3.7.3
SimPy version: 3.0.11
"""
!pip install simpy
import simpy
import random
import statistics
import numpy as np
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

wait_times = []
kitwork_times = []
small_job_count = []
med_job_count = []
big_job_count = []
small_job_days_low = 1
small_job_days_high = 6
med_job_days_low = 7
med_job_days_high = 50
big_job_days_low = 51
big_job_days_high = 85


class Implement(object):
    def __init__(self, env, num_kitters, num_custs, num_configers):
        self.env = env
        self.kitter = simpy.Resource(env, num_kitters)
        self.custs = simpy.Resource(env, num_custs)
        self.configers = simpy.Resource(env, num_configers)

        # self.usher = simpy.Resource(env, num_ushers)

     # time it takes from 1 to 3 days work
    def kitwork(self, project):
        yield self.env.timeout(random.randint(1, 3))

     # time waiting on customer from 1 to 3 days 
    def kitwait(self, project):
        yield self.env.timeout(random.randint(1, 3))

     # time waiting on configuration 
    def config_wait(self, project):
       samplerand = random.randint(1, 100)
        
       if samplerand >=1 and samplerand <= 80:
           timeout = self.env.timeout(random.randint(small_job_days_low, small_job_days_high))
           yield timeout
           small_job_count.append(timeout)    
       elif samplerand >=81 and samplerand <= 95:
          medtimeout = self.env.timeout(random.randint(med_job_days_low, med_job_days_high)) 
          yield medtimeout
          med_job_count.append(medtimeout)
       elif samplerand >=96 and samplerand <= 100:
         big_timeout =  self.env.timeout(random.randint(big_job_days_low, big_job_days_high))      
         yield  big_timeout
         big_job_count.append(big_timeout)

def go_to_implement(env, project, implementation):
    # Moviegoer arrives at the implementation
    arrival_time = env.now

    with implementation.kitter.request() as request:
        yield request
        yield env.process(implementation.kitwork(project))
    
    kitwork_times.append(env.now - arrival_time)

    with implementation.custs.request() as request:
        yield request
        yield env.process(implementation.kitwait(project))
    
    with implementation.configers.request() as request:
        yield request
        yield env.process(implementation.config_wait(project))

    wait_times.append(env.now - arrival_time)

def run_implementation(env, num_kitters, num_custs, num_configers):
    implementation = Implement( env, num_kitters, num_custs , num_configers)
   #starts with loading 1 project
    for project in range(1):
        env.process(go_to_implement(env, project, implementation))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new project

        project += 1
        env.process(go_to_implement(env, project, implementation))


def get_average_wait_time(wait_times):
    average_hours = statistics.mean(wait_times)
    average_days = average_hours
    # Pretty print the results
    # minutes, frac_minutes = divmod(average_wait, 1)
    # seconds = frac_minutes * 60
    return average_days #round(minutes), round(seconds)


# def get_user_input():
#     num_kitters = input("Input # of kitters working: ")
#     num_servers = input("Input # of servers working: ")
#     num_ushers = input("Input # of ushers working: ")
#     params = [num_kitters, num_servers, num_ushers]
#     if all(str(i).isdigit() for i in params):  # Check input is valid
#         params = [int(x) for x in params]
#     else:
#         print(
#             "Could not parse input. Simulation will use default values:",
#             "\n1 kitter, 1 server, 1 usher.",
#         )
#         params = [1, 1, 1]
#     return params


def main():
    # Setup
    random.seed(13)
    #-----------------------------------------------------
    num_kitters = 1 
    num_custs = 1
    num_configers = 10
    num_years_sim = 4
    #--------------------------------------------------

    # Run the simulation
    env = simpy.Environment()
    env.process(run_implementation(env, num_kitters, num_custs, num_configers))
    env.run(until=254 *  num_years_sim )
    
    #========================================================

    # View the results

    average_days = get_average_wait_time(wait_times)

    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(                       'Simulation Report ')     
    print(' ' )

    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('No, of Staff Fullkitting=', num_kitters)
    print('No, of Staff Configuring=', num_configers )       
    print(' ' )
    
    # print('Small job count=',len(small_job_count) ) 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++') 
    print('Percent of small jobs=', round(len(small_job_count)*100/len(wait_times),1),'%', 'of size range',small_job_days_low,'to', small_job_days_high,' days work')
    print('Percent of medium jobs=', round(len(med_job_count)*100/len(wait_times),1),'%', 'of size range',med_job_days_low,'to', med_job_days_high,' days work')
    print('Percent of big jobs=', round(len(big_job_count)*100/len(wait_times),1),'%', 'of size range',big_job_days_low,'to', big_job_days_high,' days work')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(' ') 
    
    
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++') 
    print(
        "Running simulation...over",num_years_sim,'years',
        f"\nThe average project 'end to end' time is {round(average_days)} working days to complete the projects",
    )
    print('Number of projects  processed=', len(wait_times))

    print('Average no. of projects per year =', round(len(wait_times)/num_years_sim ))
    # print("==================wait_times =============================")
    # print(wait_times)
    # print("==================kitwork_times =============================")
    # print(kitwork_times)
    print(' ') 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
   
   
    #====================Plot histogram of wait times
    print('Histograms of project end to end times in days - average duration shown in red dashes')
    n, bins, patches = plt.hist(wait_times)
    plt.axvline(x=average_days, color='r', linestyle = 'dashed')
    # n, bins, patches = plt.hist(small_job_count)
    # plt.axvline(x=small_job_count, color='r', linestyle = 'dashed')
if __name__ == "__main__":
    main()